<a href="https://colab.research.google.com/github/pratim808/hugging_face_audio_dataset_creation_TTS/blob/main/Dataset_English_techn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

manaspkundu_youtube_links_1_path = kagglehub.dataset_download('manaspkundu/youtube-links-1')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
!pip install spleeter
!pip install yt-dlp
!pip install pydub

In [ ]:
import spleeter
from __future__ import unicode_literals
import yt_dlp
from pydub import AudioSegment
from pydub.silence import split_on_silence
import csv
from pathlib import Path
from termcolor import colored
import os
import whisper

In [ ]:
def convert_audio(audio_file):
    """
    Corrects the channels, sample rate, and sample width of the audios.
    Replaces the original audio file with the one generated.
    """
    sound = AudioSegment.from_file(audio_file)
    sound = sound.set_frame_rate(16000)
    sound = sound.set_channels(1)
    sound = sound.set_sample_width(2) # 2 corresponds to 16-bit sample width in Pydub
    sound.export(audio_file, format ="wav")

In [ ]:
ydl_opts = {
    "format": "bestaudio/best",
    "audio-format": "wav",
    "outtmpl": "audio.wav",
    "noplaylist" : True
} # customizing the downloaded audio for our needs
link_num = 1 # iterates over the links in the TXT file
links_file = "/kaggle/input/youtube-links-1/1_links.txt" # File containing links to YouTube videos

In [ ]:
ydl_opts = {
    "format": "bestaudio/best",
    "audio-format": "wav",
    "outtmpl": "audio.wav",
    "noplaylist" : True
} # customizing the downloaded audio for our needs
link_num = 25 # iterates over the links in the TXT file
links_file = "/kaggle/input/youtube-links-1/1_links.txt" # File containing links to YouTube videos

In [ ]:
# Step 5: Transcribe the audio using Whisper
model = whisper.load_model("base")  # Load the Whisper model (you can choose different sizes)

def transcribe_audio(wav_file):
    """Transcribes audio from a WAV file using the Whisper model and saves the transcript."""
    result = model.transcribe(wav_file)

    # Generate the text file name with the same base name as the audio file
    transcript_file = os.path.splitext(wav_file)[0] + ".txt"

    with open(transcript_file, "w") as f:
        f.write(result["text"])

In [ ]:
'''def create_audio_transcription_pair(wav_file, transcript_file):
    """
    Creates a dictionary with audio data and transcription from the given files.
    """
    # Load audio using librosa
    audio_array, sampling_rate = librosa.load(wav_file, sr=16000)  # Set sampling rate to 16000

    with open(transcript_file, "r") as f:
        transcription = f.read()

    return {
        'audio': {
            'path': wav_file,
            'array': audio_array.astype(np.float32),  # Convert to float32
            'sampling_rate': sampling_rate
        },
        'transcription': transcription
    }'''

In [ ]:
with open(links_file) as fp:
    for link in fp:
        print("\nStarting processing for link number ", link_num)

        # Step 1: Extract and download the audio
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:  # Replace YoutubeDL with yt_dlp.YoutubeDL
                                 ydl.download([link])
        except Exception as e:
            print(colored("Link number {} cannot be downloaded. Exception: {}".format(link_num, e), 'red'))
            continue # continue with the next link in the file

        # Step 2: Separate voice from the audio
        !spleeter separate -p spleeter:2stems -o output "/kaggle/working/audio.wav"

        # Step 3: Adjust the sampling rate, sample width, and channels
        convert_audio("/kaggle/working/output/audio/vocals.wav")

        # Step 4: Split into smaller parts
        # Step 4: Split into 4-second parts
        sound_file = AudioSegment.from_wav("/kaggle/working/output/audio/vocals.wav")
        sub_chunk_length = 4000  # 4 seconds in milliseconds

        # Calculate the number of sub-chunks
        num_sub_chunks = len(sound_file) // sub_chunk_length

        print("exporting files for link number: ", link_num)
        os.mkdir(str(link_num))  # making folder named after link number

        for j in range(num_sub_chunks):
            start_time = j * sub_chunk_length
            end_time = (j + 1) * sub_chunk_length
            sub_chunk = sound_file[start_time:end_time]
            out_file = "{0}/{0}_{1}.wav".format(link_num, j + 1)  # Start with 1
            sub_chunk.export(out_file, format="wav")
            # Transcribe each 4-second chunk
            transcribe_audio(out_file)

        # Handle remaining audio
        remaining_length = len(sound_file) % sub_chunk_length # This line and the following were improperly indented
        if remaining_length > 0:
            start_time = num_sub_chunks * sub_chunk_length
            sub_chunk = sound_file[start_time:]
            out_file = "{0}/{0}_{1}.wav".format(link_num, num_sub_chunks + 1)
            sub_chunk.export(out_file, format="wav")
            # Transcribe each 4-second chunk
            transcribe_audio(out_file)
        link_num += 1

        # deleting the redundant files gnerated for previous link to save space
        !rm -rf /kaggle/working/output
        !rm /kaggle/working/audio.wav

In [ ]:
with open(links_file) as fp:
    for link in fp:
        print("\nStarting processing for link number ", link_num)

        # Step 1: Extract and download the audio
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:  # Replace YoutubeDL with yt_dlp.YoutubeDL
                                 ydl.download([link])
        except Exception as e:
            print(colored("Link number {} cannot be downloaded. Exception: {}".format(link_num, e), 'red'))
            continue # continue with the next link in the file

        # Step 2: Separate voice from the audio
        !spleeter separate -p spleeter:2stems -o output "/kaggle/working/audio.wav"

        # Step 3: Adjust the sampling rate, sample width, and channels
        convert_audio("/kaggle/working/output/audio/vocals.wav")

        # Step 4: Split into smaller parts
        # Step 4: Split into 4-second parts
        sound_file = AudioSegment.from_wav("/kaggle/working/output/audio/vocals.wav")
        sub_chunk_length = 4000  # 4 seconds in milliseconds

        # Calculate the number of sub-chunks
        num_sub_chunks = len(sound_file) // sub_chunk_length

        print("exporting files for link number: ", link_num)
        os.mkdir(str(link_num))  # making folder named after link number

        for j in range(num_sub_chunks):
            start_time = j * sub_chunk_length
            end_time = (j + 1) * sub_chunk_length
            sub_chunk = sound_file[start_time:end_time]
            out_file = "{0}/{0}_{1}.wav".format(link_num, j + 1)  # Start with 1
            sub_chunk.export(out_file, format="wav")
            # Transcribe each 4-second chunk
            transcribe_audio(out_file)

        # Handle remaining audio
        remaining_length = len(sound_file) % sub_chunk_length # This line and the following were improperly indented
        if remaining_length > 0:
            start_time = num_sub_chunks * sub_chunk_length
            sub_chunk = sound_file[start_time:]
            out_file = "{0}/{0}_{1}.wav".format(link_num, num_sub_chunks + 1)
            sub_chunk.export(out_file, format="wav")
            # Transcribe each 4-second chunk
            transcribe_audio(out_file)
        link_num += 1

        # deleting the redundant files gnerated for previous link to save space
        !rm -rf /kaggle/working/output
        !rm /kaggle/working/audio.wav

In [ ]:
'''import os
import librosa
import numpy as np
import json

def create_audio_transcription_pair(wav_file, transcript_file):
    """
    Creates a dictionary with audio data and transcription from the given files.
    """
    # Load audio using librosa
    audio_array, sampling_rate = librosa.load(wav_file, sr=16000)  # Set sampling rate to 16000

    with open(transcript_file, "r") as f:
        transcription = f.read()

    return {
        'audio': {
            'path': wav_file,
            'array': audio_array.tolist(),  # Convert to list for JSON serialization
            'sampling_rate': sampling_rate
        },
        'transcription': transcription
    }

def process_folder(folder_path, output_folder):
    """
    Processes a folder containing .wav and .txt files to create audio-transcription pairs
    and saves the output to a specified folder.
    """
    audio_transcription_pairs = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):
            wav_file = os.path.join(folder_path, filename)
            transcript_file = os.path.join(folder_path, filename.replace(".wav", ".txt"))
            if os.path.exists(transcript_file):
                pair = create_audio_transcription_pair(wav_file, transcript_file)
                audio_transcription_pairs.append(pair)
            else:
                print(f"Warning: No corresponding transcript found for {filename}")

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save the pairs to a JSON file
    output_file = os.path.join(output_folder, "audio_transcription_pairs.json")
    with open(output_file, "w") as f:
        json.dump(audio_transcription_pairs, f, indent=4)

    print(f"Audio-transcription pairs saved to {output_file}")

# Example usage:
folder_path = "/kaggle/working/1"  # Replace with the actual path to your folder
output_folder = "output_data"  # Replace with your desired output folder
process_folder(folder_path, output_folder)

# Load the saved JSON file
output_file = os.path.join(output_folder, "audio_transcription_pairs.json")
with open(output_file, "r") as f:
    pairs = json.load(f)

# Now you have a list of dictionaries, each containing audio and transcription data
for pair in pairs:
    print("Audio path:", pair['audio']['path'])
    print("Transcription:", pair['transcription'])
    print("-" * 20)'''

In [ ]:
import os
import shutil

def copy_files_to_new_folder(source_dir, dest_dir):
  """Copies files from numbered folders (1, 2, 3...) in the source directory
  to a new destination directory.

  Args:
    source_dir: The path to the directory containing the numbered folders.
    dest_dir: The path to the new directory where files will be copied.
  """

  # Create the destination directory if it doesn't exist
  os.makedirs(dest_dir, exist_ok=True)

  for folder_name in os.listdir(source_dir):
    folder_path = os.path.join(source_dir, folder_name)

    # Check if it's a directory (and you might want to add a check if it's a number)
    # Check if it's a directory AND if the folder name is a number
    if os.path.isdir(folder_path) and folder_name.isdigit():
      for filename in os.listdir(folder_path):
        source_file = os.path.join(folder_path, filename)
        destination_file = os.path.join(dest_dir, filename)

        # Copy the file to the destination directory
        shutil.copy2(source_file, destination_file)

# Set the source and destination directories
source_directory = '/kaggle/working'
destination_directory = '/kaggle/working/all_files'  # Create a new folder named 'all_files'

# Copy the files
copy_files_to_new_folder(source_directory, destination_directory)

In [ ]:
import os
import csv

def create_metadata_csv(folder_path, output_csv):
  """
  Creates a metadata.csv file with the given structure for .wav files in a folder.

  Args:
    folder_path: The path to the folder containing the .wav files.
    output_csv: The path to the output metadata.csv file.
  """

  with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["file_name", "transcription"])  # Write header row

    for filename in os.listdir(folder_path):
      if filename.endswith(".wav"):
        wav_file_path = os.path.join("data", filename)  # Construct the 'data/...' path
        transcript_file_path = os.path.join(folder_path, filename.replace(".wav", ".txt"))

        if os.path.exists(transcript_file_path):
          with open(transcript_file_path, "r") as f:
            transcription = f.read()
          writer.writerow([wav_file_path, transcription])
        else:
          print(f"Warning: No corresponding transcript found for {filename}")

# Example usage:
folder_path = "/kaggle/working/all_files"  # Replace with the actual path to your folder
output_csv = "metadata1-40.csv"
create_metadata_csv(folder_path, output_csv)

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/working/metadata1-40.csv')
print(df)

In [ ]:
import os
import shutil

def organize_audio_data(source_folder, dest_folder="data"):
  """
  Organizes audio data by creating a new folder named 'data' and copying .wav files into it.

  Args:
    source_folder: The path to the folder containing the .wav and .txt files.
    dest_folder: The name of the new folder to create (default: "data").
  """

  # Create the destination folder if it doesn't exist
  os.makedirs(dest_folder, exist_ok=True)

  for filename in os.listdir(source_folder):
    if filename.endswith('.wav'):
      source_path = os.path.join(source_folder, filename)
      dest_path = os.path.join(dest_folder, filename)
      shutil.copy2(source_path, dest_path)  # Copy the .wav file to the data folder

if __name__ == "__main__":
  source_folder = '/kaggle/working/all_files'  # Replace with the actual path to your folder
  organize_audio_data(source_folder)

In [ ]:
import shutil

shutil.rmtree('/kaggle/working/my_dataset')

In [ ]:
import os

def create_dataset_structure(base_dir):
  """
  Creates the dataset directory structure in the specified base directory.

  Args:
    base_dir: The base directory where the structure will be created.
  """

  # Create the main dataset directory
  dataset_dir = os.path.join(base_dir, "my_dataset")
  os.makedirs(dataset_dir, exist_ok=True)

  # Create subdirectories
  os.makedirs(os.path.join(dataset_dir, "data"), exist_ok=True)

  # Create README.md file
  readme_path = os.path.join(dataset_dir, "README.md")
  with open(readme_path, "w") as f:
    f.write("# My Dataset\n\nThis is a description of my dataset.\n")

  # Create metadata.csv file (you can populate this later)
  metadata_path = os.path.join(dataset_dir, "metadata.csv")
  with open(metadata_path, "w") as f:
    f.write("file,transcription\n")  # Add header row

# Example usage in Kaggle:
base_dir = "/kaggle/working"
create_dataset_structure(base_dir)

In [ ]:
import shutil
import os

def move_data_to_dataset(source_dir, target_dir):
  """
  Moves the contents of the source directory to the target directory.

  Args:
    source_dir: The path to the source directory (/kaggle/working/data).
    target_dir: The path to the target directory (/kaggle/working/my_dataset/data).
  """

  # Create the target directory if it doesn't exist
  os.makedirs(target_dir, exist_ok=True)

  for filename in os.listdir(source_dir):
    source_path = os.path.join(source_dir, filename)
    target_path = os.path.join(target_dir, filename)

    # Move the file
    shutil.move(source_path, target_path)

# Example usage in Kaggle:
source_dir = "/kaggle/working/data"
target_dir = "/kaggle/working/my_dataset/data"
move_data_to_dataset(source_dir, target_dir)

In [ ]:
import shutil

def move_metadata_to_dataset(source_path, target_path):
  """
  Moves the metadata.csv file to the target directory.

  Args:
    source_path: The path to the source metadata.csv file (/kaggle/working/metadata.csv).
    target_path: The path to the target metadata.csv file (/kaggle/working/my_dataset/metadata.csv).
  """

  shutil.move(source_path, target_path)

# Example usage in Kaggle:
source_path = "/kaggle/working/metadata1-40.csv"
target_path = "/kaggle/working/my_dataset/metadata.csv"
move_metadata_to_dataset(source_path, target_path)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("audiofolder", data_dir="/kaggle/working/my_dataset")
dataset["train"][0]

In [ ]:
!pip install datasets

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
dataset.push_to_hub("Shabdobhedi/english_technical")

In [ ]:
import os

def count_files_in_folders(directory):
  """Counts the total number of files in numbered folders (1-24) within a directory.

  Args:
    directory: The path to the directory containing the numbered folders.

  Returns:
    The total number of files found in the folders.
  """

  total_files = 0
  for i in range(1, 40):  # Loop through folders 1 to 24
    folder_path = os.path.join(directory, str(i))
    if os.path.isdir(folder_path):
      total_files += len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
  return total_files

# Get the directory path (replace with your actual directory)
directory = '/kaggle/working'

# Count the files
file_count = count_files_in_folders(directory)
print(f"Total files in folders 1-24: {file_count}")

In [ ]:
from datasets import load_dataset

ds = load_dataset("Shabdobhedi/english_technical", split ="train")

In [ ]:
print(ds)

In [ ]:
len(ds)